In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# going to the repository of this note book
%cd drive/My Drive/Thesis/CPNN_PiggyBack/PiggyBack/datasets/datasets/weather
!ls

/content/drive/My Drive/Thesis/CPNN_PiggyBack/PiggyBack/datasets/datasets/weather
us-usda-gov			    weather_mountain_ta_st124_2341.csv
weather_create_dataset.ipynb	    weather_mountain_ta_st124.csv
weather_mountain_ta_st124_1234.csv  weather_mountain_ta_st125.csv
weather_mountain_ta_st124_1432.csv  weather_mountain_ta_st125_pretrain.csv
weather_mountain_ta_st124_2143.csv


In [ ]:
from plot_utils import *

ModuleNotFoundError: No module named 'plot_utils'

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn import preprocessing

In [ ]:
station = "125"
target_col = "T_a"

In [ ]:
rootpath = "us-usda-gov/zip-file-3"
filename = f"weather_data_{station}.txt"
data = pd.read_csv(os.path.join(rootpath, filename), sep=",")
data = data.drop(columns = ["Date_time", "WY", "Year", "Month", "Day", "Hour", "Minute", "S_i"])

In [ ]:
len(data)

96432

In [ ]:
data

,T_a,RH,e_a,T_d,w_s,w_d
0,13.7,0.34,533,-1.6,0.7,255.5
1,13.1,0.36,543,-1.4,0.9,240.7
2,12.4,0.39,562,-1.0,0.7,142.0
3,12.1,0.41,579,-0.7,0.6,6.5
4,12.7,0.41,602,-0.2,0.6,332.6
...,...,...,...,...,...,...
96427,8.5,0.65,692,1.7,1.4,314.0
96428,6.1,0.75,666,1.1,1.7,264.0
96429,4.9,0.78,642,0.7,1.8,271.0
96430,5.1,0.78,650,0.8,3.5,282.0


### Data viz

In [ ]:
def plot_features (trainDF, num_feat=None):
    if num_feat is None:
        num_feat = list(trainDF.select_dtypes("int"))+list(trainDF.select_dtypes("float64"))
        for c in ["class", "label", "target"]:
            if c in num_feat:
                num_feat.remove(c)
    fig, ax = plt.subplots(nrows=len(num_feat), ncols=3, figsize=(30, 10*(len(trainDF.columns)-1)), sharex=False)
    for i,f in enumerate(num_feat):
        if type(trainDF[f].iloc[0]) == np.float64:
            ax[i][0].hist(trainDF[f])
            sns.kdeplot(trainDF[f], ax=ax[i][2])
        else:
            vc = trainDF[f].value_counts().reset_index().rename(columns={"index":"value",f:"count"}).sort_values("value")
            ax[i][0].bar(vc["value"], vc["count"])
            if f not in num_feat:
                ax[i][0].set_xticklabels(list(vc["value"]), rotation="vertical")
        for j in range(0,3):
            ax[i][j].set_title(f)
        if f in num_feat:
            sns.boxplot(data=trainDF, y=f, ax=ax[i][1])

In [ ]:
plot_features(data)

NameError: name 'plt' is not defined

In [ ]:
plt.figure(figsize=(12, 12))
corr = data.corr()
plt.matshow(corr, cmap='viridis', fignum=1)
plt.xticks(range(len(corr.columns)), corr.columns, fontsize=10, rotation=65, ha='left')
plt.yticks(range(len(corr.columns)), corr.columns, fontsize=10)
for (i, j), z in np.ndenumerate(corr):
    plt.text(j, i, '{:0.1f}'.format(z), ha='center', va='center')
plt.colorbar()
plt.show()

NameError: name 'plt' is not defined

In [ ]:
periods = 10
for i in range(1, periods+1):
    data[f"{target_col}_MA{i}"] = data[target_col].rolling(i).mean()

In [ ]:
periods = 10
for i in range(1, periods+1):
    data[f"{target_col}_lag{i}"] = data[target_col].shift(axis=0, periods=i)

In [ ]:
#data[f'{target_col}'].rolling(window=10).quantile(0.25)
periods = 10
for i in range(1, periods+1):
    data[f"{target_col}_50Q{i}"] = data[target_col].rolling(window=i).quantile(0.5)
data

,T_a,RH,T_d,w_s,w_d,T_a_25Q1,T_a_25Q2,T_a_25Q3,T_a_25Q4,T_a_25Q5,...,T_a_50Q1,T_a_50Q2,T_a_50Q3,T_a_50Q4,T_a_50Q5,T_a_50Q6,T_a_50Q7,T_a_50Q8,T_a_50Q9,T_a_50Q10
0,13.7,0.34,-1.6,0.7,255.5,13.7,NaN,NaN,NaN,NaN,...,13.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13.1,0.36,-1.4,0.9,240.7,13.1,13.250,NaN,NaN,NaN,...,13.1,13.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12.4,0.39,-1.0,0.7,142.0,12.4,12.575,12.75,NaN,NaN,...,12.4,12.75,13.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12.1,0.41,-0.7,0.6,6.5,12.1,12.175,12.25,12.325,NaN,...,12.1,12.25,12.4,12.75,NaN,NaN,NaN,NaN,NaN,NaN
4,12.7,0.41,-0.2,0.6,332.6,12.7,12.250,12.25,12.325,12.4,...,12.7,12.40,12.4,12.55,12.7,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96427,8.5,0.65,1.7,1.4,314.0,8.5,8.950,9.40,9.850,10.3,...,8.5,9.40,10.3,10.75,10.8,10.90,10.8,10.55,10.4,10.35
96428,6.1,0.75,1.1,1.7,264.0,6.1,6.700,7.30,7.900,8.5,...,6.1,7.30,8.5,9.40,10.3,10.55,10.8,10.55,10.3,10.35
96429,4.9,0.78,0.7,1.8,271.0,4.9,5.200,5.50,5.800,6.1,...,4.9,5.50,6.1,7.30,8.5,9.40,10.3,10.55,10.3,10.25
96430,5.1,0.78,0.8,3.5,282.0,5.1,4.950,5.00,5.050,5.1,...,5.1,5.00,5.1,5.60,6.1,7.30,8.5,9.40,10.3,10.25


In [ ]:
data = data.iloc[:].drop(columns="e_a")

KeyError: "['e_a'] not found in axis"

In [ ]:
data

,T_a,RH,T_d,w_s,w_d,T_a_25Q1,T_a_25Q2,T_a_25Q3,T_a_25Q4,T_a_25Q5,...,T_a_75Q1,T_a_75Q2,T_a_75Q3,T_a_75Q4,T_a_75Q5,T_a_75Q6,T_a_75Q7,T_a_75Q8,T_a_75Q9,T_a_75Q10
0,13.7,0.34,-1.6,0.7,255.5,13.7,NaN,NaN,NaN,NaN,...,13.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13.1,0.36,-1.4,0.9,240.7,13.1,13.250,NaN,NaN,NaN,...,13.1,13.550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12.4,0.39,-1.0,0.7,142.0,12.4,12.575,12.75,NaN,NaN,...,12.4,12.925,13.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12.1,0.41,-0.7,0.6,6.5,12.1,12.175,12.25,12.325,NaN,...,12.1,12.325,12.75,13.250,NaN,NaN,NaN,NaN,NaN,NaN
4,12.7,0.41,-0.2,0.6,332.6,12.7,12.250,12.25,12.325,12.4,...,12.7,12.550,12.55,12.800,13.1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96427,8.5,0.65,1.7,1.4,314.0,8.5,8.950,9.40,9.850,10.3,...,8.5,9.850,10.80,11.225,11.2,11.150,11.10,11.050,11.0,10.95
96428,6.1,0.75,1.1,1.7,264.0,6.1,6.700,7.30,7.900,8.5,...,6.1,7.900,9.40,10.550,11.2,11.100,11.10,11.050,11.0,10.95
96429,4.9,0.78,0.7,1.8,271.0,4.9,5.200,5.50,5.800,6.1,...,4.9,5.800,7.30,8.950,10.3,10.975,11.00,11.050,11.0,10.95
96430,5.1,0.78,0.8,3.5,282.0,5.1,4.950,5.00,5.050,5.1,...,5.1,5.050,5.60,6.700,8.5,9.850,10.75,10.900,11.0,10.95


In [ ]:
data["target"] = data[[f"{target_col}_50Q10", f"{target_col}_MA10"]].apply(lambda x : int(x[f"{target_col}_MA10"]<x[f"{target_col}_50Q10"]), axis=1)

data

,T_a,RH,T_d,w_s,w_d,T_a_25Q1,T_a_25Q2,T_a_25Q3,T_a_25Q4,T_a_25Q5,...,T_a_50Q1,T_a_50Q2,T_a_50Q3,T_a_50Q4,T_a_50Q5,T_a_50Q6,T_a_50Q7,T_a_50Q8,T_a_50Q9,T_a_50Q10
0,13.7,0.34,-1.6,0.7,255.5,13.7,NaN,NaN,NaN,NaN,...,13.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13.1,0.36,-1.4,0.9,240.7,13.1,13.250,NaN,NaN,NaN,...,13.1,13.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12.4,0.39,-1.0,0.7,142.0,12.4,12.575,12.75,NaN,NaN,...,12.4,12.75,13.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12.1,0.41,-0.7,0.6,6.5,12.1,12.175,12.25,12.325,NaN,...,12.1,12.25,12.4,12.75,NaN,NaN,NaN,NaN,NaN,NaN
4,12.7,0.41,-0.2,0.6,332.6,12.7,12.250,12.25,12.325,12.4,...,12.7,12.40,12.4,12.55,12.7,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96427,8.5,0.65,1.7,1.4,314.0,8.5,8.950,9.40,9.850,10.3,...,8.5,9.40,10.3,10.75,10.8,10.90,10.8,10.55,10.4,10.35
96428,6.1,0.75,1.1,1.7,264.0,6.1,6.700,7.30,7.900,8.5,...,6.1,7.30,8.5,9.40,10.3,10.55,10.8,10.55,10.3,10.35
96429,4.9,0.78,0.7,1.8,271.0,4.9,5.200,5.50,5.800,6.1,...,4.9,5.50,6.1,7.30,8.5,9.40,10.3,10.55,10.3,10.25
96430,5.1,0.78,0.8,3.5,282.0,5.1,4.950,5.00,5.050,5.1,...,5.1,5.00,5.1,5.60,6.1,7.30,8.5,9.40,10.3,10.25


In [ ]:
#data["target"] = data[[f"{target_col}_MA5", f"{target_col}_MA10"]].apply(lambda x : int(x[f"{target_col}_MA5"]<x[f"{target_col}_MA10"]), axis=1)

In [ ]:
data['target']

0        0
1        0
2        0
3        0
4        0
        ..
96427    0
96428    1
96429    1
96430    1
96431    1
Name: target, Length: 96432, dtype: int64

In [ ]:
(data["target"].value_counts() / len(data) * 100).round(2)

0    50.56
1    49.44
Name: target, dtype: float64

In [ ]:
data

,T_a,RH,T_d,w_s,w_d,T_a_25Q1,T_a_25Q2,T_a_25Q3,T_a_25Q4,T_a_25Q5,...,T_a_50Q1,T_a_50Q2,T_a_50Q3,T_a_50Q4,T_a_50Q5,T_a_50Q6,T_a_50Q7,T_a_50Q8,T_a_50Q9,T_a_50Q10
0,13.7,0.34,-1.6,0.7,255.5,13.7,NaN,NaN,NaN,NaN,...,13.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13.1,0.36,-1.4,0.9,240.7,13.1,13.250,NaN,NaN,NaN,...,13.1,13.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12.4,0.39,-1.0,0.7,142.0,12.4,12.575,12.75,NaN,NaN,...,12.4,12.75,13.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12.1,0.41,-0.7,0.6,6.5,12.1,12.175,12.25,12.325,NaN,...,12.1,12.25,12.4,12.75,NaN,NaN,NaN,NaN,NaN,NaN
4,12.7,0.41,-0.2,0.6,332.6,12.7,12.250,12.25,12.325,12.4,...,12.7,12.40,12.4,12.55,12.7,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96427,8.5,0.65,1.7,1.4,314.0,8.5,8.950,9.40,9.850,10.3,...,8.5,9.40,10.3,10.75,10.8,10.90,10.8,10.55,10.4,10.35
96428,6.1,0.75,1.1,1.7,264.0,6.1,6.700,7.30,7.900,8.5,...,6.1,7.30,8.5,9.40,10.3,10.55,10.8,10.55,10.3,10.35
96429,4.9,0.78,0.7,1.8,271.0,4.9,5.200,5.50,5.800,6.1,...,4.9,5.50,6.1,7.30,8.5,9.40,10.3,10.55,10.3,10.25
96430,5.1,0.78,0.8,3.5,282.0,5.1,4.950,5.00,5.050,5.1,...,5.1,5.00,5.1,5.60,6.1,7.30,8.5,9.40,10.3,10.25


In [ ]:

filename = "weather_mountain_ta_st125.csv"
data2 = pd.read_csv(filename, sep=",")
#data2 = data.drop(columns = ["Date_time", "WY", "Year", "Month", "Day", "Hour", "Minute", "S_i"])
data2

,RH,T_d,w_s,w_d,task1,task2,task3,task4
0,0.22,-0.003831,0.185567,0.102222,1,1,0,0
1,0.22,-0.015326,0.247423,0.835833,0,1,1,0
2,0.20,-0.022989,0.216495,0.115000,1,1,0,0
3,0.18,-0.019157,0.226804,0.271389,1,1,0,0
4,0.20,-0.015326,0.185567,0.096944,0,1,1,0
...,...,...,...,...,...,...,...,...
96417,0.65,0.065134,0.144330,0.872222,0,0,1,1
96418,0.75,0.042146,0.175258,0.733333,0,0,1,1
96419,0.78,0.026820,0.185567,0.752778,0,0,1,1
96420,0.78,0.030651,0.360825,0.783333,1,0,0,1


In [ ]:
data1=data.iloc[10:]
data1

,T_a,RH,T_d,w_s,w_d,T_a_25Q1,T_a_25Q2,T_a_25Q3,T_a_25Q4,T_a_25Q5,...,T_a_50Q1,T_a_50Q2,T_a_50Q3,T_a_50Q4,T_a_50Q5,T_a_50Q6,T_a_50Q7,T_a_50Q8,T_a_50Q9,T_a_50Q10
10,22.6,0.22,-0.1,1.8,36.8,22.6,20.275,17.60,15.175,13.6,...,22.6,21.05,19.5,17.60,15.7,14.65,13.6,13.15,12.7,12.90
11,22.2,0.22,-0.4,2.4,300.9,22.2,22.300,20.85,18.550,15.7,...,22.2,22.40,22.2,20.85,19.5,17.60,15.7,14.65,13.6,13.15
12,23.6,0.20,-0.6,2.1,41.4,23.6,22.550,22.40,21.525,19.5,...,23.6,22.90,22.6,22.40,22.2,20.85,19.5,17.60,15.7,14.65
13,25.4,0.18,-0.5,2.2,97.7,25.4,24.050,22.90,22.500,22.2,...,25.4,24.50,23.6,23.10,22.6,22.40,22.2,20.85,19.5,17.60
14,23.8,0.20,-0.4,1.8,34.9,23.8,24.200,23.70,23.250,22.6,...,23.8,24.60,23.8,23.70,23.6,23.10,22.6,22.40,22.2,20.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96427,8.5,0.65,1.7,1.4,314.0,8.5,8.950,9.40,9.850,10.3,...,8.5,9.40,10.3,10.75,10.8,10.90,10.8,10.55,10.4,10.35
96428,6.1,0.75,1.1,1.7,264.0,6.1,6.700,7.30,7.900,8.5,...,6.1,7.30,8.5,9.40,10.3,10.55,10.8,10.55,10.3,10.35
96429,4.9,0.78,0.7,1.8,271.0,4.9,5.200,5.50,5.800,6.1,...,4.9,5.50,6.1,7.30,8.5,9.40,10.3,10.55,10.3,10.25
96430,5.1,0.78,0.8,3.5,282.0,5.1,4.950,5.00,5.050,5.1,...,5.1,5.00,5.1,5.60,6.1,7.30,8.5,9.40,10.3,10.25


In [ ]:
np.round(len(data1[data1["target"].values != data2["task1"].values]) / len(data1) * 100, 2)

60.14

In [ ]:
np.round(len(data1[data1["target"].values != data2["task2"].values]) / len(data1) * 100, 2)

53.1

In [ ]:
np.round(len(data1[data1["target"].values != data2["task3"].values]) / len(data1) * 100, 2)

39.86

In [ ]:
np.round(len(data1[data1["target"].values != data2["task4"].values]) / len(data1) * 100, 2)

46.9

In [ ]:
data['target']

0        0
1        0
2        0
3        0
4        0
        ..
96427    0
96428    1
96429    1
96430    1
96431    1
Name: target, Length: 96432, dtype: int64

In [ ]:
pd.DataFrame(selected_columns)

,0,1,2,3
0,0.34,-0.061303,0.072165,0.709722
1,0.36,-0.053640,0.092784,0.668611
2,0.39,-0.038314,0.072165,0.394444
3,0.41,-0.026820,0.061856,0.018056
4,0.41,-0.007663,0.061856,0.923889
...,...,...,...,...
96427,0.65,0.065134,0.144330,0.872222
96428,0.75,0.042146,0.175258,0.733333
96429,0.78,0.026820,0.185567,0.752778
96430,0.78,0.030651,0.360825,0.783333


In [ ]:
selected_columns = data[['RH','T_d','w_s','w_d']]
selected_columns_normalized = preprocessing.MaxAbsScaler().fit_transform(selected_columns)

# Convert the normalized array back to a DataFrame with the same column names
selected_columns_normalized = pd.DataFrame(selected_columns_normalized, columns=selected_columns.columns)

selected_columns_normalized['target']=data['target']
filename = "weather_mountain_ta_st125_pretrain.csv"
selected_columns_normalized.to_csv(filename, index=False)

In [ ]:
data["task2"] = data[[target_col] + [f"{target_col}_lag{i}" for i in range(1, periods+1)]].apply(
    lambda x : int(
        x[target_col] > np.max(
            [x[c] for c in [f"{target_col}_lag{i}" for i in range(1, periods+1)]]
        )
    ),
    axis=1
)

In [ ]:
np.round(len(data[data["task2"] != data["task1"]]) / len(data) * 100, 2)

22.57

In [ ]:
(data["task1"].value_counts() / len(data) * 100).round(2)

0    55.31
1    44.69
Name: task1, dtype: float64

In [ ]:
(data["task2"].value_counts() / len(data) * 100).round(2)

0    54.9
1    45.1
Name: task2, dtype: float64

In [ ]:
data["task3"] = data["task1"].apply(lambda x : 1 if x==0 else 0)
data["task4"] = data["task2"].apply(lambda x : 1 if x==0 else 0)

In [ ]:
data_original = data.copy()

In [ ]:
lags = 10

## SCALING

In [ ]:
from sklearn import preprocessing
import os
import pickle

In [ ]:
path_write = "/Users/federicogiannini/Library/CloudStorage/OneDrive-PolitecnicodiMilano/NN4SML/datasets"

In [ ]:
data_original = (
    data_original
    .drop(
        columns=
        [target_col] +
        [f"{target_col}_lag{i}" for i in range(1, periods+1)]
    )
)

In [ ]:
tasks_config =  [[1,2,3,4], [1,4,3,2], [2,1,4,3], [2,3,4,1]]

In [ ]:
X = preprocessing.MaxAbsScaler().fit_transform(data_original.iloc[:,:-1])

In [ ]:
data_scaled = pd.DataFrame(X, columns=data_original.columns[:-1])
for i in range(1,5):
    data_scaled[f"task{i}"] = list(data_original[f"task{i}"])

In [ ]:
data_scaled.to_csv(os.path.join(path_write, f"weather_mountain_{target_col.replace('_','').lower()}_st{station}.csv"), index=False)

In [ ]:
size = len(data_scaled)
task_size = int(len(data_scaled)/4)
for tasks in tasks_config:
    dfs = []
    task = 0
    for i in range(0, size, task_size):
        df = (
            data_scaled.iloc[i:i+task_size]
            .rename(columns={f"task{tasks[task]}":"target"})
            .drop(columns=[f"task{i}" for i in range(1,5) if i!= tasks[task]])
        )
        df["task"] = task+1
        dfs.append(df)
        task += 1
        if task==4:
            break
    pd.concat(dfs).to_csv(
        os.path.join(path_write, f"weather_mountain_{target_col.replace('_','').lower()}_st{station}_{''.join([str(t) for t in tasks])}.csv"),
        index=False
    )